In [123]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

In [157]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [158]:
chatbot_data = pd.read_csv('./train.csv')
chatbot_data.head(3)

,Q,A
0,보관만 하는데 9만 원인가요 락커는 얼마죠,대형은 28만 원 중형은 19만 원 소형은 15만 원입니다
1,#@소속#이요 이름이 정겹네요,그 식당에서 드셔 보시면 홍천하면 보리밥부터 생각나실 거예요
2,#@소속# 공원 근처에 맛있는 분식집 추천해 주세요,국물 떡볶이가 있는 #@소속#을 추천해 드립니다


In [159]:
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [160]:
# 데이터 개수
len(question), len(answer)

(33994, 33994)

In [186]:
question = question[:500]
answer = answer[:500]

for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관 만 하는데 9만 원 인가요 락커 는 얼마 죠
A : 대형 은 28만 원 중형 은 19만 원 소형 은 15만 원 입니다

Q : #@ 소속 # 이요 이름 이 정겹네요
A : 그 식당 에서 드셔 보시 면 홍천 하면 보리밥 부터 생각나실 거 예요

Q : #@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요
A : 국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다

Q : 실내 에도 다양하게 준비 되어 있네요
A : 그러니 직접 체험 해 보시길 바랍니다

Q : 이번 에 김포 여행 을 가려고 하는데요
A : 무엇 을 도 와 드릴 까요



In [219]:
question

['보관 만 하는데 9만 원 인가요 락커 는 얼마 죠',
 '#@ 소속 # 이요 이름 이 정겹네요',
 '#@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요',
 '실내 에도 다양하게 준비 되어 있네요',
 '이번 에 김포 여행 을 가려고 하는데요',
 '아무 것 도 없는 줄 알았어요',
 '#@ 소속 # 맞습니다',
 '배 를 타는 시간 이 있나요',
 '절매 나루터 에서 진 탄 나루터 까지 하는 코스 맞지요',
 '#@ 소속 # 에서 받으면 되는 거 죠',
 '그건 따로 못 보긴 했는데 수요일 만 쉬는 건가 요',
 '영월 10 경 중 에 사람 이 지은 건축물 같은 거 말구요 강가 따라서 풍경 볼 수 있는 곳 추천 좀 해주세요',
 '#@ 소속 # 예약 할 수 있나 확인 하려구요',
 '녹두 한 방 백숙 은 가격 이 얼마 예요',
 '건물 뒤 에 있어서 주차장 이 안 보였군요 앞 에 차 들 은 뭐 예요',
 '체험 할 수 있는 프로그램 이 있습니까',
 '친절한 안내 감사 드립니다',
 '강습 은 받지 않고 장비 렌탈 만 이용 할 수 는 없나요',
 '땀 이 많은 편이 라 복장 때문 에 문의 드려요 등산 복 긴 팔 에 구스 경량 패딩 을 입고 갈려고 하 거든요',
 '제 가 관심 은 있는데 전혀 할 줄 몰라서요',
 '제 이메일 이 요 #@ 계정 # 입니다',
 '40분 이나 기다려야 하네요',
 '수상스키 를 체험 하고 싶은데요 금액 이 어떻게 되나요',
 '이천 여행 중 인데 맛집 을 찾고 있어요 영업 시간 이 어떻게 되나요',
 '안녕하세요 저희 가 오늘 여행 마지막 날인데요 불꽃 축제 보고 엑스포역 에서 바로 기차 를 타야 하 거든요',
 '진해 군항제 를 다녀와야겠네요',
 '엑스포 공원 갔다가 돌 머리 해수욕장 들러서 해수 찜 하러 갈거에요',
 '방금 예약 했던 게 취소 가 되서 확인 좀 부탁 드리려 구요',
 '주말 에 단독 으로 요트 투어 가 가능하나요',
 '대표 메뉴 가 무엇 인가요',
 '초등학생 체험학습 도 있나요',
 '공룡

In [187]:
# 형태소 분석 함수
def pos_tag(sentences):

    tagger = Okt()

    # 문장 품사 변수 초기화
    sentences_pos = []

    # 모든 문장 반복
    for sentence in sentences:
        sentence = re.sub(RE_FILTER, "", sentence)

        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [188]:
question = pos_tag(question)
answer = pos_tag(answer)

In [189]:
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관 만 하는데 9만 원 인가요 락커 는 얼마 죠
A : 대형 은 28만 원 중형 은 19만 원 소형 은 15만 원 입니다

Q : #@ 소속 # 이요 이름 이 정겹네요
A : 그 식당 에서 드셔 보시 면 홍천 하면 보리밥 부터 생각나실 거 예요

Q : #@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요
A : 국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다

Q : 실내 에도 다양하게 준비 되어 있네요
A : 그러니 직접 체험 해 보시길 바랍니다

Q : 이번 에 김포 여행 을 가려고 하는데요
A : 무엇 을 도 와 드릴 까요



In [190]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

In [191]:
# 길이가 0인 단어 삭제
words = [word for word in words if len(word) > 0]

In [192]:
# 중복된 단어 삭제
words = list(set(words))

In [193]:
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [194]:
len(words)

827

In [195]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '돌',
 '피로',
 '참하셔야',
 '가능',
 '군항제',
 '선택',
 '고도',
 '카',
 '이동',
 '보긴',
 '심하면',
 '드실',
 '정겹네요',
 '보세요',
 '신라',
 '하시고',
 '이에요',
 '엑스포역',
 '여행',
 '전번',
 '기다려야',
 '알았어요',
 '4시간',
 '레저',
 '부터',
 '연',
 '모듬회',
 '애견',
 '없으실까',
 '가능하시구요',
 '8시',
 '가평',
 '상품',
 '보니',
 '석양',
 '바다',
 '할',
 '복장',
 '모두',
 '일반',
 '걸릴',
 '궁금하실까',
 '사격',
 '다섯',
 '6시',
 '카페',
 '맛있는',
 '버스',
 '게',
 '사이즈',
 '제',
 '식사',
 '대표',
 '쫄깃',
 '힐링',
 'MTB',
 '전체',
 '일상생활',
 '해',
 '지는',
 '열시',
 '소형',
 '하시는거',
 '더',
 '보고',
 '않고',
 '7',
 '2',
 '만들기',
 '추가',
 '개',
 '거든요',
 '방',
 '기타',
 '2시간',
 '봄',
 '혜택',
 '이라는',
 '적',
 '대해',
 '못',
 '팔고',
 '있나요',
 '드릴려고요',
 '지금',
 '뷔페',
 '되세요',
 '설명',
 '전혀',
 '있던데요',
 '가야',
 '3만',
 '하시면',
 '이번',
 '아무',
 '프로그램',
 '8천',
 '하십시오',
 '꼭',
 '해당',
 '들',
 '생긴',
 '것',
 '지리산',
 '연락',
 '여쭤',
 '든',
 '저렴한',
 '되며',
 '소속',
 '맵',
 '정육',
 '야경',
 '양주시',
 '하려고',
 '고요',
 '을',
 '오나요',
 '18000원',
 '자산',
 '건물',
 '고객',
 '경우',
 '발자국',
 '산악',
 '방영',
 '반려동물',
 '부분',
 '닭갈비',
 '환승',
 '그

In [196]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [197]:
# 단어 => 인덱스
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '돌': 4,
 '피로': 5,
 '참하셔야': 6,
 '가능': 7,
 '군항제': 8,
 '선택': 9,
 '고도': 10,
 '카': 11,
 '이동': 12,
 '보긴': 13,
 '심하면': 14,
 '드실': 15,
 '정겹네요': 16,
 '보세요': 17,
 '신라': 18,
 '하시고': 19,
 '이에요': 20,
 '엑스포역': 21,
 '여행': 22,
 '전번': 23,
 '기다려야': 24,
 '알았어요': 25,
 '4시간': 26,
 '레저': 27,
 '부터': 28,
 '연': 29,
 '모듬회': 30,
 '애견': 31,
 '없으실까': 32,
 '가능하시구요': 33,
 '8시': 34,
 '가평': 35,
 '상품': 36,
 '보니': 37,
 '석양': 38,
 '바다': 39,
 '할': 40,
 '복장': 41,
 '모두': 42,
 '일반': 43,
 '걸릴': 44,
 '궁금하실까': 45,
 '사격': 46,
 '다섯': 47,
 '6시': 48,
 '카페': 49,
 '맛있는': 50,
 '버스': 51,
 '게': 52,
 '사이즈': 53,
 '제': 54,
 '식사': 55,
 '대표': 56,
 '쫄깃': 57,
 '힐링': 58,
 'MTB': 59,
 '전체': 60,
 '일상생활': 61,
 '해': 62,
 '지는': 63,
 '열시': 64,
 '소형': 65,
 '하시는거': 66,
 '더': 67,
 '보고': 68,
 '않고': 69,
 '7': 70,
 '2': 71,
 '만들기': 72,
 '추가': 73,
 '개': 74,
 '거든요': 75,
 '방': 76,
 '기타': 77,
 '2시간': 78,
 '봄': 79,
 '혜택': 80,
 '이라는': 81,
 '적': 82,
 '대해': 83,
 '못': 84,
 '팔고': 85,
 '있나요': 86,
 '드릴려고요': 87

In [198]:
# 인덱스 => 단어
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '돌',
 5: '피로',
 6: '참하셔야',
 7: '가능',
 8: '군항제',
 9: '선택',
 10: '고도',
 11: '카',
 12: '이동',
 13: '보긴',
 14: '심하면',
 15: '드실',
 16: '정겹네요',
 17: '보세요',
 18: '신라',
 19: '하시고',
 20: '이에요',
 21: '엑스포역',
 22: '여행',
 23: '전번',
 24: '기다려야',
 25: '알았어요',
 26: '4시간',
 27: '레저',
 28: '부터',
 29: '연',
 30: '모듬회',
 31: '애견',
 32: '없으실까',
 33: '가능하시구요',
 34: '8시',
 35: '가평',
 36: '상품',
 37: '보니',
 38: '석양',
 39: '바다',
 40: '할',
 41: '복장',
 42: '모두',
 43: '일반',
 44: '걸릴',
 45: '궁금하실까',
 46: '사격',
 47: '다섯',
 48: '6시',
 49: '카페',
 50: '맛있는',
 51: '버스',
 52: '게',
 53: '사이즈',
 54: '제',
 55: '식사',
 56: '대표',
 57: '쫄깃',
 58: '힐링',
 59: 'MTB',
 60: '전체',
 61: '일상생활',
 62: '해',
 63: '지는',
 64: '열시',
 65: '소형',
 66: '하시는거',
 67: '더',
 68: '보고',
 69: '않고',
 70: '7',
 71: '2',
 72: '만들기',
 73: '추가',
 74: '개',
 75: '거든요',
 76: '방',
 77: '기타',
 78: '2시간',
 79: '봄',
 80: '혜택',
 81: '이라는',
 82: '적',
 83: '대해',
 84: '못',
 85: '팔고',
 86: '있나요',
 87: '드릴려고요'

In [199]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [200]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)
x_encoder[0]

array([371, 174, 406, 820, 647, 581, 379, 454, 349, 755,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [201]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)
x_decoder[0]

array([  1, 422, 193, 780, 647, 505, 193, 259, 647,  65, 193, 554, 647,
       689,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [202]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)
y_decoder[0]

array([422, 193, 780, 647, 505, 193, 259, 647,  65, 193, 554, 647, 689,
         2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [203]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)), dtype=np.int8)

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [204]:
encoder_inputs = layers.Input(shape=(None,))

encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

In [205]:
# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [206]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [207]:
# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

In [208]:
# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [209]:
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [210]:
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
accuracy : 0.6816666722297668
loss : 1.5673245191574097
1/1 [==============================] - 0s 364ms/step
고객 님 님 에 에 이 # 입니다 입니다 

Total Epoch : 2
accuracy : 0.7836666703224182
loss : 0.9955487251281738
1/1 [==============================] - 0s 24ms/step
고객 에 # #@ #@ 소속 # 입니다 있습니다 입니다 있습니다 

Total Epoch : 3
accuracy : 0.9213333129882812
loss : 0.522385835647583
1/1 [==============================] - 0s 28ms/step
고객 에 # #@ #@ 소속 # 입니다 추천 드립니다 드립니다 

Total Epoch : 4
accuracy : 0.965666651725769
loss : 0.22889117896556854
1/1 [==============================] - 0s 16ms/step
#@ 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다 

Total Epoch : 5
accuracy : 0.9753333330154419
loss : 0.11507028341293335
1/1 [==============================] - 0s 25ms/step
저희 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다 

Total Epoch : 6
accuracy : 0.9816666841506958
loss : 0.0671263337135315
1/1 [==============================] - 0s 25ms/step
저희 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다 

Total Epoch : 7
accuracy : 0.9900000095367432
loss : 

In [211]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL) 

In [212]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [213]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [214]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [226]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('문의 좀 드릴려고')
input_seq

array([[265, 171,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [227]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 12ms/step


'고객 님 어떤 부분 궁금하실까 요 '